In [ ]:
!/opt/bin/nvidia-smi

Thu Feb  2 16:27:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


load dataset

In [ ]:
import re
import pandas as pd
import numpy as np
import tqdm
import nltk
import csv
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def filter(docs):
    n = len(docs)
    bek = []
    for i in range(n):
        if "\n" == docs[i]:
            bek.append(i)

    docs = [docs[i] for i in range(n) if (i not in bek)]
    return docs

In [167]:
#Load data
dataset_path = '/content/drive/MyDrive/Notebooks/data/'
with open(dataset_path + 'SwedishCorpus.txt') as f:
    lines = f.readlines()

docs = filter(lines)

# Creating a python list of URLs:
with open(dataset_path + 'articlelinks.csv', newline='') as f:
    reader = csv.reader(f)
    article_urls = list(reader)
    article_urls.reverse() # original order was from the newest

# Trimming the list of articles to reduce computation time in further steps:
url_list = []
date = []
for coronaday in article_urls:
    for newsarticle in coronaday[1:]:
        if 'lokalt' not in newsarticle: # excluding articles with local (regional) news
            url_list.append(newsarticle)
            date.append(coronaday[0])
            
url_list = url_list[:2251] # Only for one year - until 17th Jan 2021
date = date[:2251]
len(url_list) # final number of articles
    
len(article_urls) # number of days captured
len(docs)

2251

In [168]:
def specification_date(date:list):
    date_time = []
    for time in date:
        time = time.split(" ")
        if time[1] == "januari":
            time[1] = '1'
        elif time[1] == "februari":
            time[1] = '2'
        elif time[1] == "mars":
            time[1] = '3'
        elif time[1] == "april":
            time[1] = '4'
        elif time[1] == "maj":
            time[1] = '5'
        elif time[1] == "juni":
            time[1] = '6'
        elif time[1] == "juli":
            time[1] = '7'
        elif time[1] == "augusti":
            time[1] = '8'
        elif time[1] == "september":
            time[1] = '9'
        elif time[1] == "oktober":
            time[1] = '10'
        elif time[1] == "november":
            time[1] = '11'
        elif time[1] == "december":
            time[1] = '12' 
        time = '-'.join(time)
        date_time.append(time)
    return date_time

date = specification_date(date)

In [169]:
date = pd.to_datetime(date,format='%d-%m-%y', errors='coerce')

In [170]:
# Stop Words:
stop = stopwords.words('swedish') 

# Expanding by a list of 150 most common words in this corpus (meaningful words manually deleted) (.csv from SketchEngine):
with open(dataset_path + 'stopwords.csv', newline ='',encoding ='utf-8') as f:
    reader = csv.reader(f)
    extrastops = list(reader)
    
for ex in extrastops:
    stop.append(ex[0])
    
stop_words = set(stop)

In [171]:
# Tokenization
tokenizer = nltk.RegexpTokenizer(r"\w+") # deleting punctuation

tokenized_corpus = []
for article in docs:
    # Deleting a standard phrase about SVT policy, which occurs in the end of every article:
    article = article.replace("SVT:s nyheter ska stå för saklighet och opartiskhet. Det vi publicerar ska vara sant och relevant. Vid akuta nyhetslägen kan det vara svårt att få alla fakta bekräftade, då ska vi berätta vad vi vet – och inte vet. Läs mer","")
    article = article.lower()
    article = tokenizer.tokenize(article)
    article = [word for word in article if word not in stop_words and not word.isdigit()]
    tokenized_corpus.append(article)

corpus = tokenized_corpus

In [172]:
#Joining back the articles to str to comply with the initial input type for Stanza Model:
docs = []
for article in corpus:
    out_str = " "
    docs.append(out_str.join(article))

bertopic-unsup

In [ ]:
!pip install bertopic
from bertopic import BERTopic

In [176]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

umap_model = UMAP(n_neighbors=20, n_components=15, min_dist=0.0, metric='cosine') # UMAP is an technique for dimensionality reduction

#cluster_model = KMeans(n_clusters=7)
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # HDBSCAN used to cluster our documents into clusters of similar documents
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english") # Using the CountVectorizer to extract all possible words 

In [174]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(docs)

In [177]:
# The n_gram_range parameter relates to the number of words that we want in our topic representation. 
# For example, "New" and "York" are two seperate words but are often used as "New York" which represents an n-gram of 2. 
# The top_n_words refers to the number of words per topic that we want extracted.
# The min_topic_size is used to specify what the minimum size of a topic can be. The lower this value the more topics are created. 
# The nr_topics specifies, after training the topic model, the number of topics that will be reduced to.
# The calculate_probabilities calculate the probabilities of each topic to each document.

topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, 
                       calculate_probabilities= True, low_memory= False,n_gram_range = (1, 2), nr_topics="auto")

topics, probs = topic_model.fit_transform(docs, embeddings)

In [178]:
freq = topic_model.get_topic_info(); freq.head(20)

Topic  Count                                      Name
0      -1    329    -1_coronaviruset_personer_covid_länder
1       0    820     0_sverige_regeringen_procent_personer
2       1    594  1_coronaviruset_viruset_personer_sverige
3       2    163            2_patienter_covid_personer_iva
4       3    110            3_vaccin_vaccinet_fas_personer
5       4     84           4_usa_trump_donald_donald trump
6       5     40              5_tester_veckan_testa_testas
7       6     32    6_johnson_boris_boris johnson_cummings
8       7     30              7_iran_turkiet_landet_israel
9       8     29   8_antikroppar_immun_infektion_immunitet
10      9     20    9_italien_italienska_milano_lombardiet

In [179]:
topic_model.visualize_documents(docs,hide_annotations=True)

In [180]:
fig = topic_model.visualize_topics(); fig

In [182]:
topics_over_time = topic_model.topics_over_time(docs=docs, 
                                                timestamps=date, 
                                                global_tuning=True, 
                                                evolution_tuning=True
                                                )

ValueError: ignored

In [183]:
docs[0]

'coronaviruset tagit människoliv utanför kina upptäckts oron växer inför kinas största resehelg miljard kineser beräknas resande fot staden smittan först utbröt upprättat karantän flygplats tågstationen feber resa coronaviruset ursprungligen upptäcktes kina upptäckts thailand japan hittills personer dött smittats kinesiska nyåret antågande växer oron lungviruset sprida utvecklas liknar sarsepidemi tog människoliv år sen ulrika bergsten plats peking myndigheterna vidtagit omfattande åtgärder begränsa eventuell smittspridning lärt viruset fiskemarknanden stängd resa wuhan uppvisar sjukdomstecken lokaliserat personer kontakt smittade länge sett sjukdomen smittar människor viss oro viruset mutera wallensten biträdande statsepidemiolog anser risken smittspridning nuläget verkar låg nej hittills handlat personer smittade kina sen rest länderna kunnat spridits vidare länderna morgonstudion sarsepidemin kom kina mörkat information verkar förändrats wallensten annorlunda arbete delar informatio

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)